In [2]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix and Jaskaran Bains
-- Create date: 2025-04-12
-- Description: Creates the DbSecurity schema
-- =============================================
CREATE PROCEDURE [Project2].[CreateSchemaDbSecurity]
AS
BEGIN
    SET NOCOUNT ON;


    -- Create the Process schema if it doesn't exist
    IF NOT EXISTS (SELECT 1 FROM sys.schemas WHERE name = 'DbSecurity')
    BEGIN
        -- Use dynamic SQL to isolate CREATE SCHEMA in its own batch
        EXEC sp_executesql N'CREATE SCHEMA DbSecurity';
    END
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure CreateSchemaDbSecurity, Line 6
There is already an object named 'CreateSchemaDbSecurity' in the database.

Total execution time: 00:00:00.013

In [3]:
EXEC [Project2].[CreateSchemaDbSecurity];

Commands completed successfully.

Total execution time: 00:00:00.020

In [4]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix 
-- Create date: 2025-04-12
-- Description: Creates the Process schema
-- =============================================
CREATE PROCEDURE [Project2].[CreateSchemaProcess]
AS
BEGIN
    SET NOCOUNT ON;

    -- Create the Process schema if it doesn't exist
    IF NOT EXISTS (SELECT 1 FROM sys.schemas WHERE name = 'Process')
    BEGIN
        -- Use dynamic SQL to isolate CREATE SCHEMA in its own batch
        EXEC sp_executesql N'CREATE SCHEMA Process';
    END
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure CreateSchemaProcess, Line 6
There is already an object named 'CreateSchemaProcess' in the database.

Total execution time: 00:00:00.008

In [5]:
EXEC [Project2].[CreateSchemaProcess];

Commands completed successfully.

Total execution time: 00:00:00.005

In [6]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 2025-04-12
-- Description: Creates the DbSecurity.UserAuthorization table
-- =============================================
CREATE PROCEDURE [Project2].[CreateUserAuthorizationTable]
AS
BEGIN
    SET NOCOUNT ON;


    -- Drop the table if it exists
    IF EXISTS (SELECT 1 FROM INFORMATION_SCHEMA.TABLES 
               WHERE TABLE_SCHEMA = 'DbSecurity' AND TABLE_NAME = 'UserAuthorization')
    BEGIN
        DROP TABLE DbSecurity.UserAuthorization;
    END;

    -- Create the DbSecurity.UserAuthorization table
    CREATE TABLE DbSecurity.UserAuthorization (
        UserAuthorizationKey INT NOT NULL PRIMARY KEY,
        ClassTime NCHAR(5) NULL CHECK (ClassTime IN ('10:45')),
        IndividualProject NVARCHAR(60) NULL DEFAULT 'PROJECT 2 RECREATE THE BICLASS DATABASE STAR SCHEMA',
        GroupMemberLastName NVARCHAR(35) NOT NULL,
        GroupMemberFirstName NVARCHAR(25) NOT NULL,
        GroupName NVARCHAR(20) NOT NULL,
        DateAdded DATETIME2 NULL DEFAULT SYSDATETIME(),
        DateOfLastUpdate DATETIME2 NULL DEFAULT SYSDATETIME()
    );

    -- Insert initial data
    INSERT INTO DbSecurity.UserAuthorization (
        UserAuthorizationKey,
        ClassTime,
        IndividualProject,
        GroupMemberLastName,
        GroupMemberFirstName,
        GroupName,
        DateAdded,
        DateOfLastUpdate
    )
    VALUES
        (1, '10:45', 'create store procedures', 'Bains', 'Jasky', 'Group#1', DEFAULT, DEFAULT),
        (2, '10:45', 'drop foreign keys', 'Vathada', 'Mai', 'Group#1', DEFAULT, DEFAULT),
        (3, '10:45', 'load data', 'Garnica', 'Richard', 'Group#1', DEFAULT, DEFAULT),
        (4, '10:45', 'truncate tables', 'Wang', 'William', 'Group#1', DEFAULT, DEFAULT),
        (5, '10:45', 'Create first tables', 'Felix', 'Ashly', 'Group#1', DEFAULT, DEFAULT),
        (6, '10:45', 'load data', 'Cristobal', 'David', 'Group#1', DEFAULT, DEFAULT),
        (7, '10:45', 'default', 'default', 'default', 'default', DEFAULT, DEFAULT);

END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure CreateUserAuthorizationTable, Line 6
There is already an object named 'CreateUserAuthorizationTable' in the database.

Total execution time: 00:00:00.013

In [25]:
EXEC [Project2].[CreateUserAuthorizationTable];

: Msg 3726, Level 16, State 1, Procedure Project2.CreateUserAuthorizationTable, Line 16
Could not drop object 'DbSecurity.UserAuthorization' because it is referenced by a FOREIGN KEY constraint.

: Msg 2714, Level 16, State 6, Procedure Project2.CreateUserAuthorizationTable, Line 20
There is already an object named 'UserAuthorization' in the database.

Total execution time: 00:00:00.011

In [23]:
SET ANSI_NULLS ON
GO 
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix 
-- Create date: 2025-04-12
-- Description: Create the [Process].WorkflowSteps Table
-- =============================================
CREATE PROCEDURE [Project2].[CreateWorkFlowTable]
AS
BEGIN
    SET NOCOUNT ON;
    DECLARE @StartTime DATETIME2 = SYSDATETIME();
    
CREATE TABLE [Process].WorkflowSteps (
    WorkFlowStepKey INT NOT NULL PRIMARY KEY,
    WorkFlowStepDescription NVARCHAR(100) NOT NULL,
    WorkFlowStepTableRowCount INT NULL DEFAULT 0,
    StartingDateTime DATETIME2 NULL DEFAULT SYSDATETIME(),
    EndingDateTime DATETIME2 NULL DEFAULT SYSDATETIME(),
    ClassTime VARCHAR(5) NULL DEFAULT '10:45',
    UserAuthorizationKey INT NOT NULL,
    CONSTRAINT FK_WorkflowSteps_UserAuthorization FOREIGN KEY (UserAuthorizationKey) 
        REFERENCES DbSecurity.UserAuthorization (UserAuthorizationKey)
);

END;
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.011

In [24]:
EXEC [Project2].[CreateWorkFlowTable];

Commands completed successfully.

Total execution time: 00:00:00.012

In [47]:
-- =============================================
-- Author: Ashly Felix
-- Procedure: [Process].usp_TrackWorkFlow
-- Create date: 2025-04-12
-- Description: Logs a workflow step into [Process].WorkflowSteps with description, row count, and timing details.
-- =============================================
CREATE PROCEDURE [Process].[usp_TrackWorkFlow]
    @StartTime DATETIME2,
    @WorkFlowDescription NVARCHAR(100),
    @WorkFlowStepTableRowCount INT,
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    -- Insert workflow step details
    INSERT INTO [Process].[WorkflowSteps] (
        WorkFlowStepKey,
        WorkFlowStepDescription,
        WorkFlowStepTableRowCount,
        StartingDateTime,
        EndingDateTime,
        ClassTime,
        UserAuthorizationKey
    )
    VALUES (
        (SELECT ISNULL(MAX(WorkFlowStepKey), 0) + 1 FROM [Process].WorkflowSteps),
        @WorkFlowDescription,
        @WorkFlowStepTableRowCount,
        @StartTime,
        SYSDATETIME(),
        (SELECT ClassTime FROM DbSecurity.UserAuthorization WHERE UserAuthorizationKey = @UserAuthorizationKey),
        @UserAuthorizationKey
    );
END;
GO

Commands completed successfully.

Total execution time: 00:00:00.017

In [19]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix and Jaskaran Bains
-- Create date: 2025-04-13
-- Description: Queries the [Process].WorkflowSteps table to display all logged workflow steps
-- =============================================
CREATE PROCEDURE [Process].[usp_ShowWorkflowSteps]
    @UserAuthorizationKey INT -- To log who executed this
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Select all columns from [Process].WorkflowSteps, ordered by WorkFlowStepKey
    SELECT 
        WorkFlowStepKey,
        WorkFlowStepDescription,
        WorkFlowStepTableRowCount,
        StartingDateTime,
        EndingDateTime,
        ClassTime,
        UserAuthorizationKey
    FROM [Process].WorkflowSteps
    ORDER BY WorkFlowStepKey;

    -- Log the operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Queried [Process].WorkflowSteps table to display workflow steps',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure usp_ShowWorkflowSteps, Line 6
There is already an object named 'usp_ShowWorkflowSteps' in the database.

Total execution time: 00:00:00.007

In [31]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 4/13/25
-- Description:	Drops all the stored procedures that came in the db bc we will make our own
-- =============================================
ALTER procedure [Utils].[DropProcsInCSCI331FinalProject]
        @UserAuthorizationKey INT -- To log who executed this
as
begin
    set nocount on;
    DECLARE @StartTime DATETIME2 = SYSDATETIME();
    --select concat('drop prodcedure if exists  ', schema_name(o.schema_id), '.', name)
    --from sys.objects as o
    --where o.type = 'P'
    --      and o.schema_id = 9;

    drop proc if exists Project2.Load_SalesManagers;
    drop proc if exists Project2.Load_DimProductSubcategory;
    drop proc if exists Project2.Load_DimProductCategory;
    drop proc if exists Project2.Load_DimGender;
    drop proc if exists Project2.Load_DimMaritalStatus;
    drop proc if exists Project2.Load_DimOccupation;
    drop proc if exists Project2.Load_DimOrderDate;
    drop proc if exists Project2.Load_DimTerritory;
    drop proc if exists Project2.Load_DimProduct;
    drop proc if exists Project2.Load_DimCustomer;
    drop proc if exists Project2.Load_Data;
    drop proc if exists Project2.LoadSalesManagers;
    drop proc if exists Project2.LoaDimProductSubcategory;
    drop proc if exists Project2.LoadDimProductCategory;
    drop proc if exists Project2.LoadDimGender;
    drop proc if exists Project2.LoadDimMaritalStatus;
    drop proc if exists Project2.LoadDimOccupation;
    drop proc if exists Project2.LoadDimOrderDate;
    drop proc if exists Project2.LoadDimTerritory;
    drop proc if exists Project2.LoadDimProduct;
    drop proc if exists Project2.LoadDimCustomer;
    drop proc if exists Project2.LoadData;
    drop proc if exists Project2.TruncateStarSchemaData;
    drop proc if exists Project2.LoadStarSchemaData;

    EXEC [Process].usp_TrackWorkFlow
         @StartTime = @StartTime,
         @WorkFlowDescription = 'Dropped all stored procedures in Project2',
         @WorkFlowStepTableRowCount = 0,
         @UserAuthorizationKey = @UserAuthorizationKey;
end;
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.010

In [32]:
EXEC [Utils].[DropProcsInCSCI331FinalProject] 1;

Commands completed successfully.

Total execution time: 00:00:00.032

In [33]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Jaskaran Bains
-- Create date: 2025-04-13
-- Description: Creates the PkSequence schema if it does not already exist. This schema is used to store all sequence objects for surrogate keys.
-- =============================================
CREATE PROCEDURE [Project2].[CreatePkSequenceSchema]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Check if schema exists, create if missing
    IF NOT EXISTS (
        SELECT 1 
        FROM sys.schemas 
        WHERE name = 'PkSequence'
    )
    BEGIN
        EXEC('CREATE SCHEMA PkSequence');
    END

    -- Log operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Checked and created schema: PkSequence for sequence object storage',
        @WorkFlowStepTableRowCount = 0,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure CreatePkSequenceSchema, Line 6
There is already an object named 'CreatePkSequenceSchema' in the database.

Total execution time: 00:00:00.008

In [34]:
EXEC [Project2].[CreatePkSequenceSchema] 1;

Commands completed successfully.

Total execution time: 00:00:00.004

In [35]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 4/12/25
-- Description: Creates the DimProductCategory table
-- =============================================
CREATE PROCEDURE [Project2].[CreateProductCategory]
    @UserAuthorizationKey INT -- To log who executed this
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Drop the table if it exists
    IF EXISTS (SELECT 1 FROM INFORMATION_SCHEMA.TABLES 
               WHERE TABLE_SCHEMA = 'CH01-01-Dimension' AND TABLE_NAME = 'DimProductCategory')
    BEGIN
        DROP TABLE [CH01-01-Dimension].DimProductCategory;
    END;

    -- Create the DimProductCategory table
    CREATE TABLE [CH01-01-Dimension].DimProductCategory (
        ProductCategoryKey INT NOT NULL PRIMARY KEY,
        ProductCategory NVARCHAR(50) NOT NULL, -- Corrected column name to match diagram
        UserAuthorizationKey INT NOT NULL,
        DateAdded DATETIME2 NULL DEFAULT SYSDATETIME(),
        DateOfLastUpdate DATETIME2 NULL DEFAULT SYSDATETIME(),
        CONSTRAINT FK_DimProductCategory_UserAuthorization 
            FOREIGN KEY (UserAuthorizationKey) 
            REFERENCES DbSecurity.UserAuthorization (UserAuthorizationKey)
    );

    -- Log the operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Created Table DimProductCategory',
        @WorkFlowStepTableRowCount = 0,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure CreateProductCategory, Line 6
There is already an object named 'CreateProductCategory' in the database.

Total execution time: 00:00:00.009

In [36]:
EXEC [Project2].[CreateProductCategory] 1;

Commands completed successfully.

Total execution time: 00:00:00.020

In [37]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 4/12/25
-- Description: Creates the DimProductSubcategory table
-- =============================================
CREATE PROCEDURE [Project2].[CreateProductSubcategory]
    @UserAuthorizationKey INT -- To log who executed this
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Drop the table if it exists
    IF EXISTS (SELECT 1 FROM INFORMATION_SCHEMA.TABLES 
               WHERE TABLE_SCHEMA = 'CH01-01-Dimension' AND TABLE_NAME = 'DimProductSubcategory')
    BEGIN
        DROP TABLE [CH01-01-Dimension].DimProductSubcategory;
    END;

    -- Create the DimProductSubcategory table
    CREATE TABLE [CH01-01-Dimension].DimProductSubcategory (
        ProductSubcategoryKey INT NOT NULL PRIMARY KEY,
        ProductSubcategory NVARCHAR(50) NOT NULL, -- Changed from SubcategoryName to ProductSubcategory
        ProductCategoryKey INT NOT NULL,
        UserAuthorizationKey INT NOT NULL,
        DateAdded DATETIME2 NULL DEFAULT SYSDATETIME(),
        DateOfLastUpdate DATETIME2 NULL DEFAULT SYSDATETIME(),
        CONSTRAINT FK_DimProductSubcategory_ProductCategory 
            FOREIGN KEY (ProductCategoryKey) 
            REFERENCES [CH01-01-Dimension].DimProductCategory (ProductCategoryKey),
        CONSTRAINT FK_DimProductSubcategory_UserAuthorization 
            FOREIGN KEY (UserAuthorizationKey) 
            REFERENCES DbSecurity.UserAuthorization (UserAuthorizationKey)
    );

    -- Log the operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Created Table DimProductSubcategory',
        @WorkFlowStepTableRowCount = 0,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure CreateProductSubcategory, Line 6
There is already an object named 'CreateProductSubcategory' in the database.

Total execution time: 00:00:00.012

In [38]:
EXEC [Project2].[CreateProductSubcategory] 1;

Commands completed successfully.

Total execution time: 00:00:00.017

In [41]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 2025-04-12
-- Description: Adds UserAuthorizationKey column to star schema tables
-- =============================================
create PROCEDURE [Project2].[AddUserAuthorizationKeyToTables]
    @UserAuthorizationKey INT -- To log who executed this
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Add UserAuthorizationKey to dimension tables in CH01-01-Dimension schema
    ALTER TABLE [CH01-01-Dimension].DimCustomer
    ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

    ALTER TABLE [CH01-01-Dimension].DimGender
    ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

    ALTER TABLE [CH01-01-Dimension].DimMaritalStatus
    ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

    ALTER TABLE [CH01-01-Dimension].DimOccupation
    ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

    ALTER TABLE [CH01-01-Dimension].DimOrderDate
    ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

    ALTER TABLE [CH01-01-Dimension].DimProduct
    ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

    ALTER TABLE [CH01-01-Dimension].DimTerritory
    ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

    ALTER TABLE [CH01-01-Dimension].SalesManagers
    ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

    -- Add UserAuthorizationKey to fact table in CH01-01-Fact schema
    ALTER TABLE [CH01-01-Fact].Data
    ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

    -- Log the operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Added UserAuthorizationKey column to star schema tables',
        @WorkFlowStepTableRowCount = 0,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.011

In [42]:
EXEC [Project2].[AddUserAuthorizationKeyToTables] 1;

: Msg 2705, Level 16, State 4, Procedure Project2.AddUserAuthorizationKeyToTables, Line 18
Column names in each table must be unique. Column name 'UserAuthorizationKey' in table 'CH01-01-Dimension.DimGender' is specified more than once.

Total execution time: 00:00:00.006

In [46]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author:		Ashly Felix
-- Create date: 4/12/25
-- Description:	Drop the Foreign Keys From the Star Schema
-- =============================================
create PROCEDURE [Project2].[DropForeignKeysFromStarSchemaData]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

	DECLARE @StartTime DATETIME2 = SYSDATETIME();
    DECLARE @SQL NVARCHAR(MAX);

ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_SalesManagers;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimGender;
ALTER TABLE [CH01-01-Dimension].[DimProductSubcategory] DROP CONSTRAINT FK_DimProductSubcategory_ProductCategory;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimMaritalStatus;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimOccupation;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimOrderDate;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimTerritory;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimProduct;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimCustomer;
ALTER TABLE [CH01-01-Dimension].[DimProductCategory] DROP CONSTRAINT FK_DimProductCategory_UserAuthorization;
ALTER TABLE [CH01-01-Dimension].[DimProductSubcategory] DROP CONSTRAINT FK_DimProductSubcategory_UserAuthorization;

    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Dropped foreign keys from star schema',
        @WorkFlowStepTableRowCount = 0,
        @UserAuthorizationKey = @UserAuthorizationKey;

END;
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.013

In [47]:
EXEC [Project2].[DropForeignKeysFromStarSchemaData] 1;

Commands completed successfully.

Total execution time: 00:00:00.039

In [80]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 2025-04-12
-- Description: Removes identity property from star schema tables
-- =============================================
alter PROCEDURE [Project2].[RemoveIdentityFromStarSchemaTables]
    @UserAuthorizationKey INT -- To log who executed this
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Step 1: Drop all primary key constraints in Dimension and Fact schemas
    DECLARE @DropPKSql NVARCHAR(MAX);

    SELECT @DropPKSql = STRING_AGG('ALTER TABLE [' + s.name + '].[' + t.name + '] DROP CONSTRAINT [' + kc.name + ']', '; ')
    FROM sys.key_constraints kc
    JOIN sys.tables t ON kc.parent_object_id = t.object_id
    JOIN sys.schemas s ON t.schema_id = s.schema_id
    WHERE kc.type = 'PK'
      AND s.name IN ('CH01-01-Dimension', 'CH01-01-Fact');

    EXEC sp_executesql @DropPKSql;

    -- Step 2: Drop identity columns and re-add plain INT columns with PKs
    -- Manual column replacement for identity removal

    -- NOTE: This step assumes the tables are truncated beforehand (to avoid data loss)

    -- SalesManagers
    ALTER TABLE [CH01-01-Dimension].SalesManagers DROP COLUMN SalesManagerKey;
    ALTER TABLE [CH01-01-Dimension].SalesManagers ADD SalesManagerKey INT NOT NULL;
    ALTER TABLE [CH01-01-Dimension].SalesManagers ADD CONSTRAINT PK_SalesManagers PRIMARY KEY (SalesManagerKey);

    -- DimOccupation
    ALTER TABLE [CH01-01-Dimension].DimOccupation DROP COLUMN OccupationKey;
    ALTER TABLE [CH01-01-Dimension].DimOccupation ADD OccupationKey INT NOT NULL;
    ALTER TABLE [CH01-01-Dimension].DimOccupation ADD CONSTRAINT PK_DimOccupation PRIMARY KEY (OccupationKey);

    -- DimOrderDate
    ALTER TABLE [CH01-01-Dimension].DimOrderDate DROP COLUMN OrderDate;
    ALTER TABLE [CH01-01-Dimension].DimOrderDate ADD OrderDate INT NOT NULL;
    ALTER TABLE [CH01-01-Dimension].DimOrderDate ADD CONSTRAINT PK_DimOrderDate PRIMARY KEY (OrderDate);

    -- DimTerritory
    ALTER TABLE [CH01-01-Dimension].DimTerritory DROP COLUMN TerritoryKey;
    ALTER TABLE [CH01-01-Dimension].DimTerritory ADD TerritoryKey INT NOT NULL;
    ALTER TABLE [CH01-01-Dimension].DimTerritory ADD CONSTRAINT PK_DimTerritory PRIMARY KEY (TerritoryKey);

    -- DimProduct
    ALTER TABLE [CH01-01-Dimension].DimProduct DROP COLUMN ProductKey;
    ALTER TABLE [CH01-01-Dimension].DimProduct ADD ProductKey INT NOT NULL;
    ALTER TABLE [CH01-01-Dimension].DimProduct ADD CONSTRAINT PK_DimProduct PRIMARY KEY (ProductKey);

    -- DimCustomer
    ALTER TABLE [CH01-01-Dimension].DimCustomer DROP COLUMN CustomerKey;
    ALTER TABLE [CH01-01-Dimension].DimCustomer ADD CustomerKey INT NOT NULL;
    ALTER TABLE [CH01-01-Dimension].DimCustomer ADD CONSTRAINT PK_DimCustomer PRIMARY KEY (CustomerKey);

    -- ProductCategories
    ALTER TABLE [CH01-01-Dimension].DimProductCategory DROP COLUMN ProductCategoryKey;
    ALTER TABLE [CH01-01-Dimension].DimProductCategory ADD ProductCategoryKey INT NOT NULL;
    ALTER TABLE [CH01-01-Dimension].DimProductCategory ADD CONSTRAINT PK_ProductCategories PRIMARY KEY (ProductCategoryKey);

    -- ProductSubcategories
    ALTER TABLE [CH01-01-Dimension].DimProductSubcategory DROP COLUMN ProductSubcategoryKey;
    ALTER TABLE [CH01-01-Dimension].DimProductSubcategory ADD ProductSubcategoryKey INT NOT NULL;
    ALTER TABLE [CH01-01-Dimension].DimProductSubcategory ADD CONSTRAINT PK_ProductSubcategories PRIMARY KEY (ProductSubcategoryKey);

    -- Fact Table: Data
    ALTER TABLE [CH01-01-Fact].Data DROP COLUMN SalesKey;
    ALTER TABLE [CH01-01-Fact].Data ADD SalesKey INT NOT NULL;
    ALTER TABLE [CH01-01-Fact].Data ADD CONSTRAINT PK_Data PRIMARY KEY (SalesKey);

    -- Step 3: Log the operation
    EXEC [Process].[usp_TrackWorkFlow]
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Removed identity property from star schema tables',
        @WorkFlowStepTableRowCount = 0,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.011

In [81]:
EXEC [Project2].[RemoveIdentityFromStarSchemaTables] 1;

Commands completed successfully.

Total execution time: 00:00:00.109

In [82]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 4/12/25
-- Description: Truncates all star schema tables
-- =============================================
Create PROCEDURE [Project2].[TruncateStarSchemaTables]
    @UserAuthorizationKey INT -- To log who executed this
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Truncate dimension tables in CH01-01-Dimension schema
    TRUNCATE TABLE [CH01-01-Dimension].[DimCustomer];
    TRUNCATE TABLE [CH01-01-Dimension].[DimGender];
    TRUNCATE TABLE [CH01-01-Dimension].[DimMaritalStatus];
    TRUNCATE TABLE [CH01-01-Dimension].[DimOccupation];
    TRUNCATE TABLE [CH01-01-Dimension].[DimOrderDate];
    TRUNCATE TABLE [CH01-01-Dimension].[DimProduct];
    TRUNCATE TABLE [CH01-01-Dimension].[DimProductCategory];
    TRUNCATE TABLE [CH01-01-Dimension].[DimProductSubcategory];
    TRUNCATE TABLE [CH01-01-Dimension].[DimTerritory];
    TRUNCATE TABLE [CH01-01-Dimension].[SalesManagers];
    TRUNCATE TABLE [CH01-01-Fact].[Data];

    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Truncated all star schema tables',
        @WorkFlowStepTableRowCount = 0,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure TruncateStarSchemaTables, Line 6
There is already an object named 'TruncateStarSchemaTables' in the database.

Total execution time: 00:00:00.007

In [83]:
EXEC [Project2].[TruncateStarSchemaTables] 1;

Commands completed successfully.

Total execution time: 00:00:00.017

In [84]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Richard Garnica
-- Create date: 2025-04-13
-- Description: Shows amount of rows in the tables
-- =============================================
create PROCEDURE [Project2].[ShowTableStatusRowCount] 
@TableStatus VARCHAR(64),
@UserAuthorizationKey INT

AS
BEGIN
	-- SET NOCOUNT ON added to prevent extra result sets from
	-- interfering with SELECT statements.
	SET NOCOUNT ON;


	select TableStatus = @TableStatus, TableName ='CH01-01-Dimension.DimCustomer', COUNT(*) FROM [CH01-01-Dimension].DimCustomer
	select TableStatus = @TableStatus, TableName ='CH01-01-Dimension.DimGender', COUNT(*) FROM [CH01-01-Dimension].DimGender
	select TableStatus = @TableStatus, TableName ='CH01-01-Dimension.DimMaritalStatus', COUNT(*) FROM [CH01-01-Dimension].DimMaritalStatus
	select TableStatus = @TableStatus, TableName ='CH01-01-Dimension.DimOccupation', COUNT(*) FROM [CH01-01-Dimension].DimOccupation
	select TableStatus = @TableStatus, TableName ='CH01-01-Dimension.DimOrderDate', COUNT(*) FROM [CH01-01-Dimension].DimOrderDate
	select TableStatus = @TableStatus, TableName ='CH01-01-Dimension.DimProduct', COUNT(*) FROM [CH01-01-Dimension].DimProduct
	select TableStatus = @TableStatus, TableName ='CH01-01-Dimension.DimProductCategory', COUNT(*) FROM [CH01-01-Dimension].DimProductCategory
	select TableStatus = @TableStatus, TableName ='CH01-01-Dimension.DimProductSubcategory', COUNT(*) FROM [CH01-01-Dimension].DimProductSubcategory
	select TableStatus = @TableStatus, TableName ='CH01-01-Dimension.DimTerritory', COUNT(*) FROM [CH01-01-Dimension].DimTerritory
	select TableStatus = @TableStatus, TableName ='CH01-01-Dimension.SalesManagers', COUNT(*) FROM [CH01-01-Dimension].SalesManagers
	select TableStatus = @TableStatus, TableName ='CH01-01-Fact.Data', COUNT(*) FROM [CH01-01-Fact].Data

	-- Log the operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Dynamically reported row counts for star schema tables',
        @WorkFlowStepTableRowCount = 0,
        @UserAuthorizationKey = @UserAuthorizationKey;
		
END
GO


Commands completed successfully.

Commands completed successfully.

: Msg 137, Level 15, State 2, Procedure ShowTableStatusRowCount, Line 31
Must declare the scalar variable "@StartTime".

Total execution time: 00:00:00.006

In [85]:
EXEC [Project2].[ShowTableStatusRowCount]
            @TableStatus = N'Row Count after truncating',
            @UserAuthorizationKey = 1;

Commands completed successfully.

Total execution time: 00:00:00.026

TableStatus,TableName,(No column name)
Row Count after truncating,CH01-01-Dimension.DimCustomer,0


TableStatus,TableName,(No column name)
Row Count after truncating,CH01-01-Dimension.DimGender,0


TableStatus,TableName,(No column name)
Row Count after truncating,CH01-01-Dimension.DimMaritalStatus,0


...

TableStatus,TableName,(No column name)
Row Count after truncating,CH01-01-Dimension.DimProductCategory,0


TableStatus,TableName,(No column name)
Row Count after truncating,CH01-01-Dimension.DimProductSubcategory,0


TableStatus,TableName,(No column name)
Row Count after truncating,CH01-01-Dimension.DimTerritory,0


TableStatus,TableName,(No column name)
Row Count after truncating,CH01-01-Dimension.SalesManagers,0


TableStatus,TableName,(No column name)
Row Count after truncating,CH01-01-Fact.Data,0


In [86]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 2025-04-13
-- Description: Loads distinct customers into DimCustomer using surrogate keys from sequence object
-- =============================================
CREATE PROCEDURE [Project2].[LoadDimCustomer]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Ensure sequence exists and restart it
    IF NOT EXISTS (
        SELECT 1 FROM sys.sequences 
        WHERE name = 'DimCustomerSequenceObject' 
          AND SCHEMA_NAME(schema_id) = 'PkSequence'
    )
    BEGIN
        CREATE SEQUENCE PkSequence.DimCustomerSequenceObject
            AS INT START WITH 1 INCREMENT BY 1 MINVALUE 1 MAXVALUE 2147483647 CACHE;
    END
    ELSE
    BEGIN
        ALTER SEQUENCE PkSequence.DimCustomerSequenceObject RESTART WITH 1;
    END;

    -- Insert distinct customers
    INSERT INTO [CH01-01-Dimension].[DimCustomer] (
        CustomerKey,
        CustomerName,
        UserAuthorizationKey
    )
    SELECT 
        NEXT VALUE FOR PkSequence.DimCustomerSequenceObject,
        d.CustomerName,
        @UserAuthorizationKey
    FROM FileUpload.OriginallyLoadedData d
    WHERE d.CustomerName IS NOT NULL
    GROUP BY d.CustomerName;

    -- Log operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Loaded DimCustomer with surrogate keys from sequence object',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure LoadDimCustomer, Line 6
There is already an object named 'LoadDimCustomer' in the database.

Total execution time: 00:00:00.014

In [87]:
EXEC [Project2].[LoadDimCustomer] 1;

Commands completed successfully.

Total execution time: 00:00:00.402

In [96]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 2025-04-13
-- Description: Loads distinct gender values into DimGender using surrogate keys from sequence object
-- =============================================
create PROCEDURE [Project2].[LoadDimGender]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Ensure sequence exists and restart it
    IF NOT EXISTS (
        SELECT 1 FROM sys.sequences 
        WHERE name = 'DimGenderSequenceObject' 
          AND SCHEMA_NAME(schema_id) = 'PkSequence'
    )
    BEGIN
        CREATE SEQUENCE PkSequence.DimGenderSequenceObject
            AS INT START WITH 1 INCREMENT BY 1 MINVALUE 1 MAXVALUE 2147483647 CACHE;
    END
    ELSE
    BEGIN
        ALTER SEQUENCE PkSequence.DimGenderSequenceObject RESTART WITH 1;
    END;

    -- Insert distinct gender values with surrogate key
    INSERT INTO [CH01-01-Dimension].[DimGender] (
        Gender,
        GenderDescription,
        UserAuthorizationKey
    )
    SELECT 
        NEXT VALUE FOR PkSequence.DimGenderSequenceObject,
        d.Gender,
        CASE 
            WHEN d.Gender = 'M' THEN 'Male'
            WHEN d.Gender = 'F' THEN 'Female'
            ELSE 'Other'
        END,
        @UserAuthorizationKey
    FROM FileUpload.OriginallyLoadedData d
    WHERE d.Gender IS NOT NULL
    GROUP BY d.Gender;

    -- Log operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Loaded DimGender with surrogate keys from sequence object',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 121, Level 15, State 1, Procedure LoadDimGender, Line 30
The select list for the INSERT statement contains more items than the insert list. The number of SELECT values must match the number of INSERT columns.

Total execution time: 00:00:00.007

In [95]:
EXEC [Project2].[LoadDimGender] 1;

: Msg 2812, Level 16, State 62, Line 1
Could not find stored procedure 'Project2.LoadDimGender'.

Total execution time: 00:00:00.004

In [92]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 2025-04-13
-- Description: Loads distinct marital statuses into DimMaritalStatus using surrogate keys from sequence object
-- =============================================
CREATE PROCEDURE [Project2].[LoadDimMaritalStatus]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Ensure sequence exists and restart it
    IF NOT EXISTS (
        SELECT 1 FROM sys.sequences 
        WHERE name = 'DimMaritalStatusSequenceObject' 
          AND SCHEMA_NAME(schema_id) = 'PkSequence'
    )
    BEGIN
        CREATE SEQUENCE PkSequence.DimMaritalStatusSequenceObject
            AS INT START WITH 1 INCREMENT BY 1 MINVALUE 1 MAXVALUE 2147483647 CACHE;
    END
    ELSE
    BEGIN
        ALTER SEQUENCE PkSequence.DimMaritalStatusSequenceObject RESTART WITH 1;
    END;

    -- Insert distinct marital statuses with surrogate key
    INSERT INTO [CH01-01-Dimension].[DimMaritalStatus] (
        MaritalStatus,
        MaritalStatusDescription,
        UserAuthorizationKey
    )
    SELECT 
        NEXT VALUE FOR PkSequence.DimMaritalStatusSequenceObject,
        d.MaritalStatus,
        CASE 
            WHEN d.MaritalStatus = 'M' THEN 'Married'
            WHEN d.MaritalStatus = 'S' THEN 'Single'
            ELSE 'Other'
        END,
        @UserAuthorizationKey
    FROM FileUpload.OriginallyLoadedData d
    WHERE d.MaritalStatus IS NOT NULL
    GROUP BY d.MaritalStatus;

    -- Log operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Loaded DimMaritalStatus with surrogate keys from sequence object',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 207, Level 16, State 1, Procedure LoadDimMaritalStatus, Line 31
Invalid column name 'MaritalStatusKey'.

Total execution time: 00:00:00.012

In [93]:
EXEC [Project2].[LoadDimMaritalStatus] 1;

: Msg 2812, Level 16, State 62, Line 1
Could not find stored procedure 'Project2.LoadDimMaritalStatus'.

Total execution time: 00:00:00.002

In [99]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 2025-04-13
-- Description: Loads distinct occupations into DimOccupation using surrogate keys via sequence object
-- =============================================
alter PROCEDURE [Project2].[LoadDimOccupation]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Ensure sequence exists and restart it
    IF NOT EXISTS (
        SELECT 1 FROM sys.sequences 
        WHERE name = 'DimOccupationSequenceObject' 
          AND SCHEMA_NAME(schema_id) = 'PkSequence'
    )
    BEGIN
        CREATE SEQUENCE PkSequence.DimOccupationSequenceObject
            AS INT START WITH 1 INCREMENT BY 1 MINVALUE 1 MAXVALUE 2147483647 CACHE;
    END
    ELSE
    BEGIN
        ALTER SEQUENCE PkSequence.DimOccupationSequenceObject RESTART WITH 1;
    END;

    -- Insert distinct occupations using surrogate keys
    INSERT INTO [CH01-01-Dimension].[DimOccupation] (
        OccupationKey,
        Occupation,
        UserAuthorizationKey
    )
    SELECT 
        NEXT VALUE FOR PkSequence.DimOccupationSequenceObject,
        d.Occupation,
        @UserAuthorizationKey
    FROM (
        SELECT DISTINCT d.Occupation
        FROM FileUpload.OriginallyLoadedData d
        WHERE d.Occupation IS NOT NULL
    ) AS UniqueOccupations;

    -- Log operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Loaded DimOccupation with surrogate keys from sequence object',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 4104, Level 16, State 1, Procedure LoadDimOccupation, Line 37
The multi-part identifier "d.Occupation" could not be bound.

Total execution time: 00:00:00.009

In [100]:
EXEC [Project2].[LoadDimOccupation] 1;

: Msg 2812, Level 16, State 62, Line 1
Could not find stored procedure 'Project2.LoadDimOccupation'.

Total execution time: 00:00:00.004

In [101]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 2025-04-13
-- Description: Loads distinct order dates into DimOrderDate using surrogate keys from sequence object
-- =============================================
CREATE PROCEDURE [Project2].[LoadDimOrderDate]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Ensure sequence exists and restart it
    IF NOT EXISTS (
        SELECT 1 FROM sys.sequences 
        WHERE name = 'DimOrderDateSequenceObject' 
          AND SCHEMA_NAME(schema_id) = 'PkSequence'
    )
    BEGIN
        CREATE SEQUENCE PkSequence.DimOrderDateSequenceObject
            AS INT START WITH 1 INCREMENT BY 1 MINVALUE 1 MAXVALUE 2147483647 CACHE;
    END
    ELSE
    BEGIN
        ALTER SEQUENCE PkSequence.DimOrderDateSequenceObject RESTART WITH 1;
    END;

    -- Insert distinct order dates
    INSERT INTO [CH01-01-Dimension].[DimOrderDate] (
        OrderDateKey,
        OrderDate,
        MonthName,
        MonthNumber,
        Year,
        UserAuthorizationKey
    )
    SELECT 
        NEXT VALUE FOR PkSequence.DimOrderDateSequenceObject,
        d.OrderDate,
        d.MonthName,
        d.MonthNumber,
        d.Year,
        @UserAuthorizationKey
    FROM FileUpload.OriginallyLoadedData d
    WHERE d.OrderDate IS NOT NULL
    GROUP BY d.OrderDate, d.MonthName, d.MonthNumber, d.Year;

    -- Log operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Loaded DimOrderDate with surrogate keys from sequence object',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 207, Level 16, State 1, Procedure LoadDimOrderDate, Line 31
Invalid column name 'OrderDateKey'.

Total execution time: 00:00:00.012

In [102]:
EXEC [Project2].[LoadDimOrderDate] 1;

: Msg 2812, Level 16, State 62, Line 1
Could not find stored procedure 'Project2.LoadDimOrderDate'.

Total execution time: 00:00:00.005

In [103]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 2025-04-13
-- Description: Loads distinct products into DimProduct with surrogate keys via sequence object
-- =============================================
alter PROCEDURE [Project2].[LoadDimProduct]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Ensure sequence exists and restart it
    IF NOT EXISTS (
        SELECT 1 FROM sys.sequences 
        WHERE name = 'DimProductSequenceObject' 
          AND SCHEMA_NAME(schema_id) = 'PkSequence'
    )
    BEGIN
        CREATE SEQUENCE PkSequence.DimProductSequenceObject
            AS INT START WITH 1 INCREMENT BY 1 MINVALUE 1 MAXVALUE 2147483647 CACHE;
    END
    ELSE
    BEGIN
        ALTER SEQUENCE PkSequence.DimProductSequenceObject RESTART WITH 1;
    END;

    -- Load distinct products by ProductCode (1st name per code)
    INSERT INTO [CH01-01-Dimension].[DimProduct] (
        ProductKey,
        ProductSubcategoryKey,
        ProductCategory,
        ProductSubcategory,
        ProductCode,
        ProductName,
        Color,
        ModelName,
        UserAuthorizationKey
    )
    SELECT 
        NEXT VALUE FOR PkSequence.DimProductSequenceObject,
        NULL AS ProductSubcategoryKey,
        ProductCategory,
        ProductSubcategory,
        ProductCode,
        ProductName,
        Color,
        ModelName,
        @UserAuthorizationKey
    FROM (
        SELECT *,
               ROW_NUMBER() OVER (PARTITION BY ProductCode ORDER BY ProductName) AS rn
        FROM FileUpload.OriginallyLoadedData
        WHERE ProductCode IS NOT NULL
    ) AS Ranked
    WHERE rn = 1;

    -- Log operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Loaded DimProduct with surrogate keys from sequence object',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 208, Level 16, State 6, Procedure LoadDimProduct, Line 6
Invalid object name 'Project2.LoadDimProduct'.

Total execution time: 00:00:00.008

In [104]:
EXEC [Project2].[LoadDimProduct] 1;

: Msg 2812, Level 16, State 62, Line 1
Could not find stored procedure 'Project2.LoadDimProduct'.

Total execution time: 00:00:00.004

In [105]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Maitri Vathada
-- Create date: 2025-04-13
-- Description: Loads distinct territory combinations into DimTerritory using surrogate keys from a sequence object
-- =============================================
CREATE PROCEDURE [Project2].[LoadDimTerritory]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Ensure sequence exists and restart it
    IF NOT EXISTS (
        SELECT 1 FROM sys.sequences 
        WHERE name = 'DimTerritorySequenceObject' 
          AND SCHEMA_NAME(schema_id) = 'PkSequence'
    )
    BEGIN
        CREATE SEQUENCE PkSequence.DimTerritorySequenceObject
            AS INT START WITH 1 INCREMENT BY 1 MINVALUE 1 MAXVALUE 2147483647 CACHE;
    END
    ELSE
    BEGIN
        ALTER SEQUENCE PkSequence.DimTerritorySequenceObject RESTART WITH 1;
    END;

    -- Insert distinct territory values
    INSERT INTO [CH01-01-Dimension].[DimTerritory] (
        TerritoryKey,
        TerritoryGroup,
        TerritoryCountry,
        TerritoryRegion,
        UserAuthorizationKey
    )
    SELECT 
        NEXT VALUE FOR PkSequence.DimTerritorySequenceObject,
        d.TerritoryGroup,
        d.TerritoryCountry,
        d.TerritoryRegion,
        @UserAuthorizationKey
    FROM (
        SELECT DISTINCT TerritoryGroup, TerritoryCountry, TerritoryRegion
        FROM FileUpload.OriginallyLoadedData
        WHERE TerritoryGroup IS NOT NULL
          AND TerritoryCountry IS NOT NULL
          AND TerritoryRegion IS NOT NULL
    ) AS d;

    -- Log operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Loaded DimTerritory with surrogate keys from sequence object',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.022

In [106]:
EXEC [Project2].[LoadDimTerritory] 1;

Commands completed successfully.

Total execution time: 00:00:00.277

In [107]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Jaskaran Bains
-- Create date: 2025-04-13
-- Description: Loads distinct sales managers into Dim table using surrogate key sequence and conditional office logic
-- =============================================
CREATE PROCEDURE [Project2].[LoadSalesManagers] 
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Ensure sequence exists and restart it
    IF NOT EXISTS (
        SELECT 1 FROM sys.sequences 
        WHERE name = 'SalesManagersSequenceObject' 
          AND SCHEMA_NAME(schema_id) = 'PkSequence'
    )
    BEGIN
        CREATE SEQUENCE PkSequence.SalesManagersSequenceObject
            AS INT START WITH 1 INCREMENT BY 1 MINVALUE 1 MAXVALUE 2147483647 CACHE;
    END
    ELSE
    BEGIN
        ALTER SEQUENCE PkSequence.SalesManagersSequenceObject RESTART WITH 1;
    END;

    -- Insert distinct sales managers
    INSERT INTO [CH01-01-Dimension].SalesManagers (
        SalesManagerKey,
        Category,
        SalesManager,
        Office,
        UserAuthorizationKey
    )
    SELECT 
        NEXT VALUE FOR PkSequence.SalesManagersSequenceObject,
        ProductCategory,
        SalesManager,
        CASE
            WHEN SalesManager LIKE 'Marco%' THEN 'Redmond'
            WHEN SalesManager LIKE 'Alberto%' THEN 'Seattle'
            WHEN SalesManager LIKE 'Maurizio%' THEN 'Redmond'
            ELSE 'Seattle'
        END,
        @UserAuthorizationKey
    FROM FileUpload.OriginallyLoadedData
    WHERE SalesManager IS NOT NULL AND ProductCategory IS NOT NULL
    GROUP BY ProductCategory, SalesManager;

    -- Log operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Loaded SalesManagers with surrogate keys from sequence object and assigned office values',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.008

In [108]:
EXEC [Project2].[LoadSalesManagers]  1;

Commands completed successfully.

Total execution time: 00:00:00.122

In [118]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Jaskaran Bains
-- Create date: 2025-04-13
-- Description: Loads distinct product categories into Dim table using surrogate key sequence
-- =============================================
create PROCEDURE [Project2].[LoadProductCategory]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Ensure sequence exists and restart it
    IF NOT EXISTS (
        SELECT 1 FROM sys.sequences 
        WHERE name = 'ProductCategoriesSequenceObject' 
          AND SCHEMA_NAME(schema_id) = 'PkSequence'
    )
    BEGIN
        CREATE SEQUENCE PkSequence.ProductCategoriesSequenceObject
            AS INT START WITH 1 INCREMENT BY 1 MINVALUE 1 MAXVALUE 2147483647 CACHE;
    END
    ELSE
    BEGIN
        ALTER SEQUENCE PkSequence.ProductCategoriesSequenceObject RESTART WITH 1;
    END;

    -- Insert distinct product categories
    INSERT INTO [CH01-01-Dimension].[DimProductCategory] (
        ProductCategoryKey,
        ProductCategory,
        UserAuthorizationKey
    )
    SELECT 
        NEXT VALUE FOR PkSequence.ProductCategoriesSequenceObject,
        d.ProductCategory,
        @UserAuthorizationKey
    FROM FileUpload.OriginallyLoadedData d
    WHERE d.ProductCategory IS NOT NULL
    GROUP BY d.ProductCategory;

    -- Log operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Loaded ProductCategories with surrogate keys from sequence object',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.014

In [119]:
EXEC [Project2].[LoadProductCategory] 1;

Commands completed successfully.

Total execution time: 00:00:00.049

In [123]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Jaskaran Bains
-- Create date: 2025-04-13
-- Description: Loads distinct product subcategories into Dim table using surrogate keys
-- =============================================
alter PROCEDURE [Project2].[LoadProductSubcategories]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Ensure sequence exists and restart it
    IF NOT EXISTS (
        SELECT 1 FROM sys.sequences 
        WHERE name = 'ProductSubcategoriesSequenceObject' 
          AND SCHEMA_NAME(schema_id) = 'PkSequence'
    )
    BEGIN
        CREATE SEQUENCE PkSequence.ProductSubcategoriesSequenceObject
            AS INT START WITH 1 INCREMENT BY 1 MINVALUE 1 MAXVALUE 2147483647 CACHE;
    END
    ELSE
    BEGIN
        ALTER SEQUENCE PkSequence.ProductSubcategoriesSequenceObject RESTART WITH 1;
    END;

    -- Insert distinct product subcategories with resolved ProductCategoryKey
    INSERT INTO [CH01-01-Dimension].[DimProductSubcategory] (
        ProductSubcategoryKey,
        ProductCategoryKey,
        ProductSubcategory,
        UserAuthorizationKey
    )
    SELECT 
        NEXT VALUE FOR PkSequence.ProductSubcategoriesSequenceObject,
        pc.ProductCategoryKey,
        d.ProductSubcategory,
        @UserAuthorizationKey
    FROM (
        SELECT DISTINCT ProductCategory, ProductSubcategory
        FROM FileUpload.OriginallyLoadedData
        WHERE ProductSubcategory IS NOT NULL AND ProductCategory IS NOT NULL
    ) d
    INNER JOIN [CH01-01-Dimension].[DimProductCategory] pc
        ON d.ProductCategory = pc.ProductCategory;

    -- Log operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Loaded ProductSubcategories with surrogate keys and foreign keys from sequence object',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.012

In [124]:
EXEC [Project2].[LoadProductSubcategories] 1;

Commands completed successfully.

Total execution time: 00:00:00.071

In [125]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 2025-04-13
-- Description: Loads fact table Data using surrogate keys from sequence object
-- =============================================
CREATE PROCEDURE [Project2].[LoadData]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Ensure sequence exists and restart it
    IF NOT EXISTS (
        SELECT 1 FROM sys.sequences 
        WHERE name = 'DataSequenceObject' 
          AND SCHEMA_NAME(schema_id) = 'PkSequence'
    )
    BEGIN
        CREATE SEQUENCE PkSequence.DataSequenceObject
            AS INT START WITH 1 INCREMENT BY 1 MINVALUE 1 MAXVALUE 2147483647 CACHE;
    END
    ELSE
    BEGIN
        ALTER SEQUENCE PkSequence.DataSequenceObject RESTART WITH 1;
    END;

    -- Insert data into the fact table by joining with dimension tables
    INSERT INTO [CH01-01-Fact].[Data] (
        SalesKey,
        SalesManagerKey,
        OccupationKey,
        TerritoryKey,
        ProductKey,
        CustomerKey,
        ProductCategory,
        SalesManager,
        ProductSubcategory,
        ProductCode,
        ProductName,
        Color,
        ModelName,
        OrderQuantity,
        UnitPrice,
        ProductStandardCost,
        SalesAmount,
        OrderDate,
        MonthName,
        MonthNumber,
        Year,
        CustomerName,
        MaritalStatus,
        Gender,
        Education,
        Occupation,
        TerritoryRegion,
        TerritoryCountry,
        TerritoryGroup,
        UserAuthorizationKey
    )
    SELECT 
        NEXT VALUE FOR PkSequence.DataSequenceObject AS SalesKey,
        sm.SalesManagerKey,
        occ.OccupationKey,
        terr.TerritoryKey,
        prod.ProductKey,
        cust.CustomerKey,
        f.ProductCategory,
        f.SalesManager,
        f.ProductSubcategory,
        f.ProductCode,
        f.ProductName,
        f.Color,
        f.ModelName,
        f.OrderQuantity,
        f.UnitPrice,
        f.ProductStandardCost,
        f.SalesAmount,
        f.OrderDate,
        f.MonthName,
        f.MonthNumber,
        f.Year,
        f.CustomerName,
        f.MaritalStatus,
        f.Gender,
        f.Education,
        f.Occupation,
        f.TerritoryRegion,
        f.TerritoryCountry,
        f.TerritoryGroup,
        @UserAuthorizationKey
    FROM FileUpload.OriginallyLoadedData f
    LEFT JOIN [CH01-01-Dimension].SalesManagers sm
        ON f.SalesManager = sm.SalesManager AND f.ProductCategory = sm.Category
    LEFT JOIN [CH01-01-Dimension].DimOccupation occ
        ON f.Occupation = occ.Occupation
    LEFT JOIN [CH01-01-Dimension].DimTerritory terr
        ON f.TerritoryRegion = terr.TerritoryRegion
        AND f.TerritoryCountry = terr.TerritoryCountry
        AND f.TerritoryGroup = terr.TerritoryGroup
    LEFT JOIN [CH01-01-Dimension].DimProduct prod
        ON f.ProductCode = prod.ProductCode
    LEFT JOIN [CH01-01-Dimension].DimCustomer cust
        ON f.CustomerName = cust.CustomerName;

    -- Log operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Loaded Data fact table with surrogate keys from sequence object',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.008

In [126]:
EXEC [Project2].[LoadData] 1;

Commands completed successfully.

Total execution time: 00:00:01.874

In [127]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix
-- Create date: 2025-04-13
-- Description: Creates the ParsedFileUpload schema and populates OriginallyLoadedData table with data from dimension tables
-- =============================================
CREATE PROCEDURE [Project2].[CreateParsedFileUpload]
    @UserAuthorizationKey INT -- To log who executed this
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Create the ParsedFileUpload schema if it doesn't exist
    IF SCHEMA_ID('ParsedFileUpload') IS NULL
    BEGIN
        EXEC sp_executesql N'CREATE SCHEMA ParsedFileUpload';
    END;

    -- Drop the table if it exists
    IF OBJECT_ID('ParsedFileUpload.OriginallyLoadedData') IS NOT NULL
    BEGIN
        DROP TABLE ParsedFileUpload.OriginallyLoadedData;
    END;

    -- Populate ParsedFileUpload.OriginallyLoadedData by joining dimension tables
    SELECT
        cust.CustomerKey,
        cust.CustomerName,
        gen.GenderKey,
        gen.Gender,
        gen.GenderDescription,
        ms.MaritalStatusKey,
        ms.MaritalStatus,
        ms.MaritalStatusDescription,
        occ.OccupationKey,
        occ.Occupation,
        od.OrderDateKey,
        od.OrderDate,
        od.MonthName,
        od.MonthNumber,
        od.Year,
        terr.TerritoryKey,
        terr.TerritoryRegion,
        terr.TerritoryCountry,
        terr.TerritoryGroup,
        sm.SalesManagerKey,
        sm.SalesManager,
        sm.Category AS ProductCategory,
        sm.Office,
        pc.ProductCategoryKey,
        pc.ProductCategory,
        psc.ProductSubcategoryKey,
        psc.ProductSubcategory,
        prod.ProductKey,
        prod.ProductCode,
        prod.ProductName,
        prod.Color,
        prod.ModelName
    INTO ParsedFileUpload.OriginallyLoadedData
    FROM [CH01-01-Dimension].DimCustomer cust
    CROSS JOIN [CH01-01-Dimension].DimGender gen
    CROSS JOIN [CH01-01-Dimension].DimMaritalStatus ms
    CROSS JOIN [CH01-01-Dimension].DimOccupation occ
    CROSS JOIN [CH01-01-Dimension].DimOrderDate od
    CROSS JOIN [CH01-01-Dimension].DimTerritory terr
    CROSS JOIN [CH01-01-Dimension].SalesManagers sm
    CROSS JOIN [CH01-01-Dimension].ProductCategories pc
    CROSS JOIN [CH01-01-Dimension].ProductSubcategories psc
    CROSS JOIN [CH01-01-Dimension].DimProduct prod;

    -- Log the operation
    EXEC [Process].usp_TrackWorkFlow
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Created ParsedFileUpload schema and populated OriginallyLoadedData table with dimension data',
        @WorkFlowStepTableRowCount = @@ROWCOUNT,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure CreateParsedFileUpload, Line 6
There is already an object named 'CreateParsedFileUpload' in the database.

Total execution time: 00:00:00.012

In [136]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Jaskaran Bains
-- Create date: 2025-04-13
-- Description:	Restores foreign keys to star schema tables
-- =============================================
alter PROCEDURE [Project2].[AddForeignKeysToStarSchemaData]
        @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Restore foreign keys in ProductSubcategories
    ALTER TABLE [CH01-01-Dimension].DimProductSubcategory
    ADD CONSTRAINT FK_ProductSubcategory_DimProductCategory
    FOREIGN KEY (ProductCategoryKey)
    REFERENCES [CH01-01-Dimension].DimProductCategory (ProductCategoryKey);

    -- Restore foreign keys in DimProduct
    ALTER TABLE [CH01-01-Dimension].DimProduct
    ADD CONSTRAINT FK_DimProduct_ProductSubcategory
    FOREIGN KEY (ProductSubcategoryKey)
    REFERENCES [CH01-01-Dimension].DimProductSubcategory (ProductSubcategoryKey);

    -- Restore foreign keys in Data
    ALTER TABLE [CH01-01-Fact].Data
    ADD CONSTRAINT FK_Data_SalesManagers
    FOREIGN KEY (SalesManagerKey)
    REFERENCES [CH01-01-Dimension].SalesManagers (SalesManagerKey);

    --ALTER TABLE [CH01-01-Fact].Data
    --ADD CONSTRAINT FK_Data_DimGender
    --FOREIGN KEY (GenderKey)
    --REFERENCES [CH01-01-Dimension].DimGender (GenderKey);

    --ALTER TABLE [CH01-01-Fact].Data
    --ADD CONSTRAINT FK_Data_DimMaritalStatus
    --FOREIGN KEY (MaritalStatusKey)
    --REFERENCES [CH01-01-Dimension].DimMaritalStatus (MaritalStatusKey);

    ALTER TABLE [CH01-01-Fact].Data
    ADD CONSTRAINT FK_Data_DimOccupation
    FOREIGN KEY (OccupationKey)
    REFERENCES [CH01-01-Dimension].DimOccupation (OccupationKey);

    ALTER TABLE [CH01-01-Fact].Data
    ADD CONSTRAINT FK_Data_DimOrderDate
    FOREIGN KEY (OrderDateKey)
    REFERENCES [CH01-01-Dimension].DimOrderDate (OrderDateKey);

    ALTER TABLE [CH01-01-Fact].Data
    ADD CONSTRAINT FK_Data_DimTerritory
    FOREIGN KEY (TerritoryKey)
    REFERENCES [CH01-01-Dimension].DimTerritory (TerritoryKey);

    ALTER TABLE [CH01-01-Fact].Data
    ADD CONSTRAINT FK_Data_DimProduct
    FOREIGN KEY (ProductKey)
    REFERENCES [CH01-01-Dimension].DimProduct (ProductKey);

    ALTER TABLE [CH01-01-Fact].Data
    ADD CONSTRAINT FK_Data_DimCustomer
    FOREIGN KEY (CustomerKey)
    REFERENCES [CH01-01-Dimension].DimCustomer (CustomerKey);

    ALTER TABLE [CH01-01-Fact].Data
    ADD CONSTRAINT FK_Data_DimProductCategory
    FOREIGN KEY (ProductCategoryKey)
    REFERENCES [CH01-01-Dimension].DimProductCategory (ProductCategoryKey);

    ALTER TABLE [CH01-01-Fact].Data
    ADD CONSTRAINT FK_Data_DimProductSubcategory
    FOREIGN KEY (ProductSubcategoryKey)
    REFERENCES [CH01-01-Dimension].DimProductSubcategory (ProductSubcategoryKey);

    -- Log the operation
    EXEC [Process].[usp_TrackWorkFlow]
        @StartTime = @StartTime,
        @WorkFlowDescription = 'Restored foreign keys to star schema tables',
        @WorkFlowStepTableRowCount = 0,
        @UserAuthorizationKey = @UserAuthorizationKey;
END;
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.010

In [137]:
EXEC [Project2].[AddForeignKeysToStarSchemaData] 1;

: Msg 2714, Level 16, State 5, Procedure Project2.AddForeignKeysToStarSchemaData, Line 15
There is already an object named 'FK_ProductSubcategory_DimProductCategory' in the database.

: Msg 1750, Level 16, State 1, Procedure Project2.AddForeignKeysToStarSchemaData, Line 15
Could not create constraint or index. See previous errors.

Total execution time: 00:00:00.007

In [138]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author: Ashly Felix and Jaskaran Bains
-- Create date: 2025-04-13
-- Description: Orchestrates the entire star schema loading process by executing all procedures in order
-- =============================================
ALTER PROCEDURE [Project2].[LoadStarSchemaData]
@UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @StartTime DATETIME2 = SYSDATETIME();

    -- Begin a transaction to ensure consistency
    BEGIN TRY
        BEGIN TRANSACTION;

        --
        -- Create schemas (logging not possible yet)
        --
        EXEC [Project2].[CreateSchemaDbSecurity];
        EXEC [Project2].[CreateSchemaProcess];

        --
        -- Create metadata tables for logging
        --
        EXEC [Project2].[CreateUserAuthorizationTable];
        EXEC [Project2].[CreateWorkFlowTable] 1;
        --EXEC [Process].[usp_TrackWorkFlows];

        --
        -- Drop all the prior stored procedures
        --
        EXEC [Utils].[DropProcsInCSCI331FinalProject] 5;


        --
        -- Create remaining schemas (logging now possible)
        --
        EXEC [Project2].[CreatePkSequenceSchema] 6;

        --
        -- Add required columns (e.g., UserAuthorizationKey) to star schema tables
        --
        EXEC [Project2].[AddUserAuthorizationKeyToTables] 1;

        --
        -- Drop all foreign keys prior to truncating tables in the star schema
        --
        EXEC [Project2].[DropForeignKeysFromStarSchemaData] 2;

        --
        -- Remove identity properties from star schema tables
        --
        EXEC [Project2].[RemoveIdentityFromStarSchemaTables] 3;

        --
        -- Check row count before truncation
        --
        EXEC [Project2].[ShowTableStatusRowCount]
            @TableStatus = N'Pre-truncate of tables',
            @UserAuthorizationKey = 4;

        --
        -- Always truncate the star schema data
        --
        EXEC [Project2].[TruncateStarSchemaTables] 5;

        --
        -- Load the star schema
        --
        EXEC [Project2].[LoadProductCategories] 6;
        EXEC [Project2].[LoadProductSubcategories] 1;
        EXEC [Project2].[LoadDimProduct] 2;
        EXEC [Project2].[LoadSalesManagers] 3;
        EXEC [Project2].[LoadDimCustomer] 4;
        EXEC [Project2].[LoadDimGender] 5;
        EXEC [Project2].[LoadDimMaritalStatus] 6;
        EXEC [Project2].[LoadDimOccupation] 1;
        EXEC [Project2].[LoadDimOrderDate] 2;
        EXEC [Project2].[LoadDimTerritory] 3;
        EXEC [Project2].[LoadData] 4;

        --
        -- Check row count after loading the star schema
        --
        EXEC [Project2].[ShowTableStatusRowCount]
            @TableStatus = N'Row Count after loading the star schema',
            @UserAuthorizationKey = 5;

        --
        -- Create ParsedFileUpload.OriginallyLoadedData with data from dimension tables
        --
        EXEC [Project2].[CreateParsedFileUpload] 6;

        --
        -- Recreate all foreign keys after loading the star schema
        --
        EXEC [Project2].[AddForeignKeysToStarSchemaData] 1;

        --
        --Show Workflow Steps 
        --
        EXEC [Process].[usp_ShowWorkflowSteps] 2;

        --
        -- Commit the transaction if all steps succeed
        --
        COMMIT TRANSACTION;

        --
        -- Log the overall operation
        --
        EXEC [Process].usp_TrackWorkFlow
            @StartTime = @StartTime,
            @WorkFlowDescription = 'Successfully executed all steps to load star schema data',
            @WorkFlowStepTableRowCount = 0,
            @UserAuthorizationKey = 3;
    END TRY
    BEGIN CATCH
        -- Rollback the transaction if any step fails
        IF @@TRANCOUNT > 0
            ROLLBACK TRANSACTION;

        -- Log the error
        DECLARE @ErrorMessage NVARCHAR(4000) = ERROR_MESSAGE();
        EXEC [Process].usp_TrackWorkFlow
            @StartTime = @StartTime,
            @WorkFlowDescription = @ErrorMessage,
            @WorkFlowStepTableRowCount = 0,
            @UserAuthorizationKey = 4;

        -- Re-throw the error to the caller
        THROW;
    END CATCH;
END;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 208, Level 16, State 6, Procedure LoadStarSchemaData, Line 6
Invalid object name 'Project2.LoadStarSchemaData'.

Total execution time: 00:00:00.007

In [140]:
EXEC [Project2].[LoadStarSchemaData] 1;

: Msg 2812, Level 16, State 62, Line 1
Could not find stored procedure 'Project2.LoadStarSchemaData'.

Total execution time: 00:00:00.004

In [142]:
EXEC [Project2].[ShowTableStatusRowCount]
            @TableStatus = N'Row Count after loading the star schema',
            @UserAuthorizationKey = 1;

Commands completed successfully.

Total execution time: 00:00:00.034

TableStatus,TableName,(No column name)
Row Count after loading the star schema,CH01-01-Dimension.DimCustomer,18400


TableStatus,TableName,(No column name)
Row Count after loading the star schema,CH01-01-Dimension.DimGender,0


TableStatus,TableName,(No column name)
Row Count after loading the star schema,CH01-01-Dimension.DimMaritalStatus,0


...

TableStatus,TableName,(No column name)
Row Count after loading the star schema,CH01-01-Dimension.DimProductCategory,3


TableStatus,TableName,(No column name)
Row Count after loading the star schema,CH01-01-Dimension.DimProductSubcategory,17


TableStatus,TableName,(No column name)
Row Count after loading the star schema,CH01-01-Dimension.DimTerritory,10


TableStatus,TableName,(No column name)
Row Count after loading the star schema,CH01-01-Dimension.SalesManagers,9


TableStatus,TableName,(No column name)
Row Count after loading the star schema,CH01-01-Fact.Data,60398
